In [0]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from glob import glob
import cv2
import json
import sys
import random
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
sys.path.append('/content/drive/My Drive/Colab Notebooks')

Using TensorFlow backend.


In [0]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

In [0]:
def encode_rle(mask):
    """Returns encoded mask (run length) as a string.
    Parameters
    ----------
    mask : np.ndarray, 2d
        Mask that consists of 2 unique values: 0 - denotes background, 1 - denotes object.
    Returns
    -------
    str
        Encoded mask.
    Notes
    -----
    Mask should contains only 2 unique values, one of them must be 0, another value, that denotes
    object, could be different from 1 (for example 255).
    """
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]

    return ' '.join(str(x) for x in runs)


def decode_rle(rle_mask, shape=(512, 512)):
    """Decodes mask from rle string.
    Parameters
    ----------
    rle_mask : str
        Run length as string formatted.
    shape : tuple of 2 int, optional (default=(320, 240))
        Shape of the decoded image.
    Returns
    -------
    np.ndarray, 2d
        Mask that contains only 2 unique values: 0 - denotes background, 255 - denotes object.
    
    """
    s = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for low, high in zip(starts, ends):
        img[low:high] = 255

    return img.reshape(shape)

Модель обучена в ноутбуку Main_Train при оптимальных параметрах(filter - 64, 1024, epoch = 10, batch_size = 10) Модель доступна по ссылке https://drive.google.com/file/d/1OZFwbD8cvRR_YyKY1NOVfMchJf0mW3ZI/view?usp=sharing

In [0]:
saved_model = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/cig_butts/M4_plus4epochs_dice_bincrosentr_f64-1024_DICE-078.h5", custom_objects = {"dice_coef" : dice_coef})

Валидационые сеты получена аналогично тренировачным в ноутбуке Main_Train. С изменением - валидационные картинки в последнем измерении были уменьшены на 1, посколько в расширении png, а натренирована сеть на jpg

In [0]:
X_val = np.load("/content/drive/My Drive/Colab Notebooks/cig_butts/X_val_uint8.npy")/255.0

In [0]:
Y_val = np.load("/content/drive/My Drive/Colab Notebooks/cig_butts/Y_val_uint8.npy")/255.0

In [0]:
Y_val_pred = saved_model.predict(X_val, batch_size=10, verbose = 0)

Функция restrict - переводит в ноль все числа в массиве меньше определенного трешхолда, а если больше - то в 255

In [0]:
def restrict(Y_val, treshold):
  X = (Y_val*255).reshape(512,512).astype(np.uint8)
  return (np.ones((512,512))*(X > treshold)).astype(np.uint8)*255

In [0]:
EPS = 1e-10


def dice(true, pred):
    """Dice score.
    Parameters
    ----------
    true : np.ndarray, 2d
         Ground truth mask that consists of 2 unique values: 0 - denotes background,
         1 - denotes object.
    pred : np.ndarray, 2d
         Predicted mask that consists of 2 unique values: 0 - denotes background,
         1 - denotes object.
    Returns
    -------
    float from 0 to 1
        Dice score. The greater the value of dice score the better.
    Notes
    -----
    Masks should contains only 2 unique values, one of them must be 0, another value, that denotes
    object, could be different from 1 (for example 255).
    """
    true = true.astype(bool)
    pred = pred.astype(bool)

    intersection = (true & pred).sum()
    im_sum = true.sum() + pred.sum()

    return 2.0 * intersection / (im_sum + EPS)

Выберем для каждой картинки оптимальный трешхолд и применим функцию restrict

In [0]:
Y_val_pred_rest = []
for h in range(200):
  B = []
  for i in range(255):
    B.append(dice((Y_val[h]*255).reshape(512,512).astype(np.uint8), restrict(Y_val_pred[h], i)))
  imax = B.index(max(B))
  Y_val_pred_rest.append(restrict(Y_val_pred[h], imax))

Сохраним полученный массив

In [0]:
np.save("/content/drive/My Drive/Colab Notebooks/cig_butts/Y_val_pred_rest", Y_val_pred_rest)

Получение фрэймворка из массива аналогично в ноутбуке Main Train

In [0]:
Final = {"img_id" : [], "rle_mask" : []}
for i in range(200):
  Qval = encode_rle((Y_val[i]*255).reshape(512,512).astype(np.uint8))
  for y in range(200):
    if Orig.iloc[:,2][y] == Qval:
      img_id = y
  Qval_pred = encode_rle(Y_val_pred_rest[i]) 
  Final["img_id"].append(img_id)
  Final["rle_mask"].append(Qval_pred)   

In [0]:
Final

In [0]:
DatFrame = pd.DataFrame.from_dict(Final)

In [0]:
DF1 = DatFrame.sort_values(by = ["img_id"])
DF1

In [0]:
Final_DF = DF1.reset_index(drop=True)

In [0]:
Final_DF

In [0]:
Final_DF.to_csv("/content/drive/My Drive/Colab Notebooks/cig_butts/pred_val.csv", sep=',', encoding='utf-8')

Сохраним финальный csv файл

In [0]:
np.save("/content/drive/My Drive/Colab Notebooks/cig_butts/val_pred_M4.npy", Y_val_pred)